In [4]:
from pydantic.v1 import BaseModel, root_validator, SecretStr

In [30]:
class User:
    def __init__(self, name:str, age:int, is_active:bool=True, password:str=''):
        self.name = name
        self.age = age
        self.is_active = is_active
        self.password = password
u1 = User(name=123, age='sss', is_active=1243, password='aaaa')
u1.__dict__

{'name': 123, 'age': 'sss', 'is_active': 1243, 'password': 'aaaa'}

In [15]:
class User(BaseModel):
    name:str
    age: int
    is_active: bool = True
    password: SecretStr = '123'
try:
    u1 = User(name=123, age='sss', is_active=1243, password='aaaa')
    u1.__dict__
except Exception as e:
    print(e)

2 validation errors for User
age
  value is not a valid integer (type=type_error.integer)
is_active
  value could not be parsed to a boolean (type=type_error.bool)


In [16]:
try:
    u1 = User()
    u1.__dict__
except Exception as e:
    print(e)

2 validation errors for User
name
  field required (type=value_error.missing)
age
  field required (type=value_error.missing)


In [17]:
u1 = User(name='john', age=20, is_active=False, password='not visible')
u1.dict()

{'name': 'john',
 'age': 20,
 'is_active': False,
 'password': SecretStr('**********')}

In [31]:
class User(BaseModel):
    name:str
    age: int
    is_active: bool = True
    password: SecretStr = ''

    @root_validator()
    def check_age(cls, values):
        age = values.get('age')
        if age < 18:
            raise ValueError("age must be at least 18")
        return values
try:
    user = User(name="Alice", age=15)
    print(user.dict()) 
except Exception as e:
    print(e)    

1 validation error for User
__root__
  age must be at least 18 (type=value_error)


In [33]:
from datetime import datetime
from pydantic import BaseModel, root_validator, ValidationError, validator, field_validator

class Event(BaseModel):
    start_date: datetime
    end_date: datetime

    @root_validator(pre=True)
    def check_dates(cls, values):
        start_date, end_date = values.get('start_date'), values.get('end_date')
        # Attempt to parse strings to datetime if not already datetime objects
        if isinstance(start_date, str):
            try:
                values['start_date'] = datetime.fromisoformat(start_date)
            except ValueError:
                raise ValueError("start_date must be a valid ISO format datetime string")
        if isinstance(end_date, str):
            try:
                values['end_date'] = datetime.fromisoformat(end_date)
            except ValueError:
                raise ValueError("end_date must be a valid ISO format datetime string")
        
        if 'start_date' in values and 'end_date' in values:
            if values['start_date'] >= values['end_date']:
                raise ValueError("end_date must be after start_date")
        return values

    @validator('start_date', 'end_date', pre=False)
    def check_future_dates(cls, v):
        if v < datetime.now():
            raise ValueError(f"{v} must be in the future")
        return v

# Example usage
try:
    event = Event(start_date="2021-09-01T15:00:00", end_date="2021-09-01T14:00:00")
except ValidationError as e:
    print(e)

try:
    event = Event(start_date="2023-09-01T15:00:00", end_date="2023-09-02T14:00:00")
    print(event)
except ValidationError as e:
    print(e)


1 validation error for Event
  Value error, end_date must be after start_date [type=value_error, input_value={'start_date': datetime.d...time(2021, 9, 1, 14, 0)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error
2 validation errors for Event
start_date
  Value error, 2023-09-01 15:00:00 must be in the future [type=value_error, input_value=datetime.datetime(2023, 9, 1, 15, 0), input_type=datetime]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error
end_date
  Value error, 2023-09-02 14:00:00 must be in the future [type=value_error, input_value=datetime.datetime(2023, 9, 2, 14, 0), input_type=datetime]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error


/tmp/ipykernel_7199/968085228.py:8: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @root_validator(pre=True)
/tmp/ipykernel_7199/968085228.py:28: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @validator('start_date', 'end_date', pre=False)


In [50]:
from pydantic import BaseModel, SecretStr, field_validator

class CustomSecretStr(SecretStr):
    @classmethod
    def __get_pydantic_json_schema__(cls, core_schema, handler):
        json_schema = handler(core_schema)
        json_schema.update(type="string", format="password")
        return json_schema

class UserProfile(BaseModel):
    username: str
    password: CustomSecretStr    
    age: int

    @field_validator('username')
    def username_alphanumeric(cls, v: str) -> str:
        if not v.isalnum():
            raise ValueError('Username must be alphanumeric')
        return v

    @field_validator('password', mode='before')
    def convert_password_to_secretstr(cls, v: str) -> CustomSecretStr:
        # Convert plain text password to CustomSecretStr if it is not already one
        if isinstance(v, str):
            return CustomSecretStr(v)
        return v

    @field_validator('password')
    def password_length(cls, v: CustomSecretStr) -> CustomSecretStr:
        if len(v.get_secret_value()) < 8:
            raise ValueError('Password must be at least 8 characters long')
        return v
# Example usage
try:
    user = UserProfile(
        username="johnDoe123",
        password="securePassword123",  # User provides plain text
        email="john@example.com",
        age=25
    )
    print(user)
except ValidationError as e:
    print(e)
    

username='johnDoe123' password=CustomSecretStr('**********') age=25


In [49]:
from pydantic import BaseModel, SecretStr, ValidationError, field_validator

class UserProfile(BaseModel):
    username: str
    password: SecretStr
    email: str
    age: int

    @field_validator('username')
    def username_alphanumeric(cls, v: str) -> str:
        if not v.isalnum():
            raise ValueError('Username must be alphanumeric')
        return v

    @field_validator('password')
    def convert_password_to_secretstr(cls, v: str) -> SecretStr:
        # Convert plain text password to SecretStr if it is not already one
        if isinstance(v, str):
            return SecretStr(v)
        return v

    @field_validator('password')
    def password_length(cls, v: SecretStr) -> SecretStr:
        if len(v.get_secret_value()) < 8:
            raise ValueError('Password must be at least 8 characters long')
        return v

    @field_validator('email')
    def email_must_contain_at(cls, v: str) -> str:
        if '@' not in v:
            raise ValueError('Email must contain @')
        return v.lower()

    @field_validator('age')
    def age_must_be_positive(cls, v: int) -> int:
        if v <= 0:
            raise ValueError('Age must be positive')
        return v

# Example usage
try:
    user = UserProfile(
        username="johnDoe123",
        password="securePassword123",  # User provides plain text
        email="john@example.com",
        age=25
    )
    print(user)
except ValidationError as e:
    print(e)


username='johnDoe123' password=SecretStr('**********') email='john@example.com' age=25


In [54]:
from pydantic import BaseModel, FieldValidationInfo, field_validator
from datetime import datetime
from typing import Optional

class User(BaseModel):
    name: str
    age: int
    signup_ts: Optional[datetime] = None

    def __init__(self, **data):
        # Handle preprocessing that doesn't involve validation
        super().__init__(**data)  # Calls the BaseModel __init__

        # Post-processing after Pydantic's validation
        if self.signup_ts is None:
            self.signup_ts = datetime.now()

    @field_validator('age')
    def check_age(cls, value: int) -> int:
        if value < 18:
            raise ValueError('Age must be at least 18')
        return value

# Example usage
user = User(name="Alice", age=30)
print(user.dict())  # Output includes 'signup_ts' set to current datetime if not provided


{'name': 'Alice', 'age': 30, 'signup_ts': datetime.datetime(2024, 8, 19, 8, 9, 38, 441333)}


In [57]:
from pydantic import BaseModel, field_validator, model_validator  ### root_vaidator has been depreciated in v2
from datetime import datetime
from typing import Optional

class User(BaseModel):
    name: str
    age: int
    signup_ts: Optional[datetime] = None

    def __init__(self, **data):
        print("Initializing the class with data:", data)
        super().__init__(**data)
        print("Running __init__ post processing")
        if self.signup_ts is None:
            self.signup_ts = datetime.now()

    @model_validator(mode="before")
    def check_root_pre(cls, values):
        print("Running model_validator with mode='before'")
        return values    

    @field_validator('name')
    def check_name(cls, value: str) -> str:
        print("Running field_validator for name")
        return value

   @field_validator('age', mode="before")
    def check_age_before(cls, value: int) -> int:
        print("Running field_validator with mode='before' for age")
        return value   

    @model_validator(mode="after")
    def check_root_post(cls, values):
        print("Running model_validator with mode='after'")
        return values

# Example usage
user = User(name="Alice", age=30)


Initializing the class with data: {'name': 'Alice', 'age': 30}
Running model_validator with mode='before'
Running field_validator for name
Running field_validator with mode='before' for age
Running model_validator with mode='after'
Running __init__ post processing
